In [3]:
! python3.10 -m pip install numpy
! python3.10 -m pip install matplotlib 

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
import json
from queue import PriorityQueue

In [5]:
class Street:
  def __init__(self, id, name):
    self.id = id
    self.name = name
    
class Point:
  def __init__(self, id, x, y, streetList: list):
    self.id = id
    self.x = x
    self.y = y
    self.streetList = streetList
  
  def getId(self) -> int:
    return self.id
  
  def getPosition(self) -> list:
    return [self.x, self.y]
  
  def getStreetList(self) -> list:
    return self.streetList

road_id = 0
class Road:
  def __init__(self , point1: Point, point2: Point, oneWayRoad):
    global road_id
    self.id = road_id
    road_id += 1
    self.point1 = point1
    self.point2 = point2
    self.oneWayRoad = oneWayRoad
  def getPoint1(self)->Point:
    return self.point1
  def getPoint2(self)->Point:
    return self.point2
  def getLength(self):
    if self.length != -1:
      x1, y1 = self.getPoint1().getPosition()
      x2, y2 = self.getPoint2().getPosition()
      self.length = ((x1-x2)**2+(y1-y2)**2)**(1/2)
    return abs()

class Graph:
  next_point_dict = {}
  def add_road(self, road_to_add: Road):
    p1Id = road_to_add.point1.getId()
    p2Id = road_to_add.point2.getId()
    if p1Id not in self.next_point_dict:
      self.next_point_dict[p1Id] = []
    if p2Id not in self.next_point_dict:
      self.next_point_dict[p2Id] = []
    if road_to_add.oneWayRoad!= 1:
      self.next_point_dict[p2Id] += [p1Id]
    if road_to_add.oneWayRoad!= -1:
      self.next_point_dict[p1Id] += [p2Id]
  def get_next_point_dict(self):
    return self.next_point_dict

In [6]:
streetDict = {}
pointDict = {}
roadDict = {}
graph = Graph()

In [7]:
def read_data_from_file():
  with open("data.json") as data:
    fileContent = data.read()
    dataLoad = json.loads(fileContent)
    
    for street_id in dataLoad["street"]:
      newStreet = Street(street_id, dataLoad["street"][street_id])
      streetDict[street_id] = newStreet
    for point_id in dataLoad["point"]:
      tmpPoint = dataLoad["point"][point_id]
      newPoint = Point(int(point_id), tmpPoint["x"], tmpPoint["y"], tmpPoint["streetList"])
      pointDict[int(point_id)] = newPoint
    
    for road in dataLoad["road"]:
      newRoad = Road(pointDict[road["point1"]], pointDict[road["point2"]], road["oneWayRoad"])
      graph.add_road(newRoad)
    print(graph.next_point_dict)
    
read_data_from_file()

{1: [], 2: [1, 5], 3: [1, 4], 4: [2], 5: [2, 6, 11, 15], 6: [5, 7], 7: [8, 13, 13], 8: [], 9: [10, 11], 10: [], 11: [5, 9], 12: [10], 13: [14, 7, 7, 14], 14: [13, 13, 19], 15: [5, 16, 17], 16: [15], 17: [15, 18], 18: [17], 19: [12, 14]}


In [8]:
def getNearestRoad(x, y)->Road:
  pass

def calculateLength(point1: Point, point2: Point):
  x1, y1 = point1.getPosition()
  x2, y2 = point2.getPosition()
  return ((x1-x2)**2+(y1-y2)**2)**(1/2)

In [9]:

def findPath(x_start, y_start, x_target, y_target):
  startRoad = getNearestRoad(x_start, y_start)
  targetRoad = getNearestRoad(x_target, y_target)
  visitedPoint = {}
  nextPointDict = graph.get_next_point_dict()
  pathPriorityQueue = PriorityQueue()
  pathPriorityQueue.put(
    ( calculateLength(startRoad.getPoint1(), 
                    Point(-1, x_start, y_start, [])),
      startRoad.getPoint1().getId()))
  pathPriorityQueue.put(
    ( calculateLength(startRoad.getPoint2(), 
                    Point(-1, x_start, y_start, [])),
      startRoad.getPoint2().getId()))
  while not pathPriorityQueue.empty():
    tempLength, tempPointId = pathPriorityQueue.get()
    if visitedPoint[tempPointId] == True:
      continue
    visitedPoint[tempPointId] = True
    for nextPointId in nextPointDict[tempPointId]:
      pathPriorityQueue.put(
        tempLength + calculateLength(
            pointDict[tempPointId], 
            pointDict[nextPointId]
          ),
        startRoad.getPoint2().getId()
      )
  